In [ ]:
import warnings
warnings.simplefilter("ignore")

# Statistics in Python: an introduction

In part adapted from: 
* Gaël Varoquaux [Scipy Lecture](https://scipy-lectures.org/packages/statistics/index.html#linear-models-multiple-factors-and-analysis-of-variance)
* The fMRI course developed by M. Brett and myself
* Many other resources


## Requirements
1. Standard scientific Python environment (numpy, scipy, matplotlib)
2. Pandas
3. [Statsmodels](https://www.statsmodels.org/stable/index.html)
4. [Seaborn](https://seaborn.pydata.org/)

**Disclaimer: Gender questions**

   Some of the examples of this tutorial are chosen around gender 
    questions. The reason is that on such questions controlling the truth
    of a claim actually matters to many people. 


## What this will not cover

* **Bayesian statistics in Python**:

    This chapter does not cover tools for Bayesian statistics.  
    Of particular interest for Bayesian modelling is 
    [PyMC](http://pymc-devs.github.io/pymc),  
    which implements a probabilistic programming language in Python.

* **Permutation testing** (see sklearn)

* **Read a statistics book**:
   The [Think stats](http://greenteapress.com/wp/think-stats-2e>)  
   book is available as free PDF or in print and is a great 
   introduction to statistics.

Data representation and interaction
====================================

Data as a table
---------------------

The setting that we consider for statistical analysis is that of multiple
*observations* or *samples* described by a set of different *attributes*
or *features*. The data can than be seen as a 2D table, or matrix, with
columns giving the different attributes of the data, and rows the
observations. For instance, the data contained in
[This csv file](https://raw.githubusercontent.com/scipy-lectures/scipy-lecture-notes/master/packages/statistics/examples/brain_size.csv)


The data in this .csv file looks like:
```
"";"Gender";"FSIQ";"VIQ";"PIQ";"Weight";"Height";"MRI_Count"
"1";"Female";133;132;124;"118";"64.5";816932
"2";"Male";140;150;124;".";"72.5";1001121
"3";"Male";139;123;150;"143";"73.3";1038437
"4";"Male";133;129;128;"172";"68.8";965353
"5";"Female";137;132;134;"147";"65.0";951545
```

## The pandas data-frame


We will store and manipulate this data in a
    `pandas.DataFrame`, from the [pandas](http://pandas.pydata.org) module. It is the Python equivalent of
    the spreadsheet table. It is different from a 2D ``numpy`` array as it
    has named columns, can contain a mixture of different data types by
    column, and has elaborate selection and pivotal mechanisms.


### Creating dataframes: reading data files or converting arrays



In [ ]:
import pandas

csvfile = ('https://raw.githubusercontent.com/scipy-lectures/'
          'scipy-lecture-notes/master/packages/statistics/examples/brain_size.csv')

data = pandas.read_csv(csvfile, sep=';', index_col=0)

data.head()

In [ ]:
print('What type is data ? : ', type(data))
print("It stores values in a numpy array:", type(data.values))
data.values[:5]

### warning: Missing values


The weight of the second individual is missing in the CSV file. In this
  particular file, missing values are represented by "`.`". If we don't specify
  the missing value (NA = not available) marker, pandas will not recognize this
  and we will not be able to do statistical analysis.


In [ ]:
data = pandas.read_csv(csvfile, sep=';', na_values=".")
data.head(3) #.dropna()

## Creating from arrays

A `pandas.DataFrame` can also be seen as a dictionary of 1D 'series', eg arrays or lists. If we have 3
``numpy`` arrays:

In [ ]:
import numpy as np
t = np.linspace(-6, 6, 20)
sin_t = np.sin(t)
cos_t = np.cos(t)

# We can expose them as a :class:`pandas.DataFrame`::
pandas.DataFrame({'t': t, 'sin': sin_t, 'cos': cos_t}).head(4)

# Question : what is the first argument ? 

## Manipulating data


In [ ]:
print('data.shape: ', data.shape)
print('data.columns: ',data.columns)  # It has columns
print('\nFemale VIQ mean: ', data[data['Gender'] == 'Female']['VIQ'].mean())

data.describe()

In [ ]:
# Question: Will we get the same results with : 
# `data.dropna().describe()` ?

In [ ]:
# "Group by"
groupby_gender = data.groupby('Gender')
for gender, value in groupby_gender['VIQ']:
    print((gender, value.mean()))

In [ ]:
type(groupby_gender)

In [ ]:
data.loc[data['Gender'] == 'Female', 'VIQ'] += 1

In [ ]:
for gender, value in groupby_gender['VIQ']:
    print((gender, value.mean()))

data['VIQ'] += 1

# Question : what if we look at the groupby_gender object ? 

In [ ]:
for gender, value in groupby_gender['VIQ']:
    print((gender, value.mean()))

* What is the mean value for VIQ for the full population?
* How many males/females were included in this study?

  **Hint** use 'tab completion' to find out the methods that can be
  called, instead of 'mean' in the above example.

* What is the average value of MRI counts expressed in log units, for
  males and females?


## Plotting data


In [ ]:
from pandas import plotting as pdplt
pdplt.scatter_matrix(data[['Weight', 'Height', 'MRI_Count']]);

In [ ]:
pdplt.scatter_matrix(data[['PIQ', 'VIQ', 'FSIQ']]);

The IQ metrics are bimodal, as if there are 2 sub-populations.

# Hypothesis testing: comparing two groups


For simple statistical tests:

https://en.wikipedia.org/wiki/Statistical_hypothesis_testing

we will use the `scipy.stats` sub-module of [scipy](http://docs.scipy.org/doc/)



In [ ]:
from scipy import stats
from scipy import stats as sst

Cohen's d effect size : 

$\hspace{3cm} d = \frac{\mu}{\sigma}$

$\mu$ the non normalized effect size, $\sigma$ the standard deviation of the **data**

Author report: APOE effect on hippocampal volume has a p value of 6.6311e-10, n=733
What is the effect size of APOE on the hippocampal volume ?


In [ ]:
# create a normal(0,1) variable 
n01 = sst.norm(0,1.)
# n01 = sst.t(df=733)

z = n01.isf(6.6311e-10)

#wat = n01.cdf(6.6311e-10)

d = n01.isf(6.6311e-10)/np.sqrt(733)
print("z = {:4.3f} d = {:4.3f}".format(z,d))

In [ ]:
n01.rvs(size=(3,3))

In [ ]:
# scipy stats has a great number of distribution, all with pdf cdf, sf, isf, etc ...  
# but you can also sample from these: 
n, start, width = 1000, 10, 20
unif10_20 = sst.uniform(loc=start,scale=width)
data_uniform = unif10_20.rvs(size=(n,))

import seaborn as sns
ax = sns.distplot(data_uniform,
                  bins=50,
#                  kde=True,
                  color='skyblue',
                  hist_kws={"linewidth": 15,'alpha':1})
ax.set(xlabel='Uniform Distribution ', ylabel='Frequency')

![title](stats-distrib.png)

## Student's t-test: the simplest statistical test


`scipy.stats.ttest_1samp` tests if the population mean of data is
likely to be equal to a given value (technically if observations are
drawn from a Gaussian distributions of given population mean). It returns
the [T statistic](https://en.wikipedia.org/wiki/Student%27s_t-test),
and the [p-value](https://en.wikipedia.org/wiki/P-value) (see the
function's help)

In [ ]:
stats.ttest_1samp(data['VIQ'], 0)

### Definition of a p-value


Probability of observing a statistic equal to the one seen 
in the data, or one that is more extreme, when the null 
hypothesis is true


Requires:
* Knowledge of the null hypothesis
* Choice of a statistic
* Concept of repeating the whole study in the same way
    - Same study design
    - Same sampling scheme
    - Same definition of the statistic

### 2-sample t-test: testing for difference across populations

We have seen above that the mean VIQ in the  male and  
female populations were different. To test if this is  
significant, we do a 2-sample t-test with  

`scipy.stats.ttest_ind`



In [ ]:
np.set_printoptions(precision=4) # he non !

female_viq = data[data['Gender'] == 'Female']['VIQ']
male_viq = data[data['Gender'] == 'Male']['VIQ']
stats.ttest_ind(female_viq, male_viq)   

#stats.ttest_ind?

### Paired tests: repeated measurements on the same individuals

PIQ, VIQ, and FSIQ give 3 measures of IQ.  
Let us test if FISQ and PIQ are significantly  
different. We can use a 2 sample test


In [ ]:
# Box plots of different columns for each gender
groupby_gender.boxplot(column=['FSIQ', 'VIQ', 'PIQ'])
groupby_gender.groups

In [ ]:
stats.ttest_ind(data['FSIQ'], data['PIQ'])

# Can you see a problem with this approach ?

The problem with this approach is that it forgets that there are links between  
observations: FSIQ and PIQ are measured on the same individuals. Thus the  
variance due to inter-subject variability is confounding, and can be removed,  
using a "paired test", or ["repeated measures test"](https://en.wikipedia.org/wiki/Repeated_measures_design)  

In [ ]:
# considering that FSIQ and PIQ come from the same participants:

stats.ttest_rel(data['FSIQ'], data['PIQ'])

In [ ]:
# This is equivalent to a 1-sample test on the difference:

stats.ttest_1samp(data['FSIQ'] - data['PIQ'], 0)

# stats.ttest_1samp?

T-tests assume Gaussian errors.  

We can use a [Wilcoxon signed-rank test](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test) that relaxes this assumption:


In [ ]:
stats.wilcoxon(data['FSIQ'], data['PIQ'])

### Exercise

* Test the difference between weights in males and females.
* Use non parametric statistics to test the difference  
  between VIQ in males and females.

In [ ]:
import pandas
from statsmodels.formula.api import ols

data = pandas.read_csv(csvfile, sep=';', na_values='.')

model = ols('VIQ ~ Gender + MRI_Count + Height', data).fit()
print(model.summary())

# Here, we don't need to define a contrast, as we are testing a single
# coefficient of our model, and not a combination of coefficients.
# However, defining a contrast, which would then be a 'unit contrast',
# will give us the same results
print(model.f_test([0, 1, 0, 0]))

In [ ]:
###############################################################################
# Here we plot a scatter matrix to get intuitions on our results.
# This goes beyond what was asked in the exercise

# This plotting is useful to get an intuitions on the relationships between
# our different variables

from pandas.tools import plotting
import matplotlib.pyplot as plt

# Fill in the missing values for Height for plotting
data['Height'].fillna(method='pad', inplace=True)

# The parameter 'c' is passed to plt.scatter and will control the color
# The same holds for parameters 'marker', 'alpha' and 'cmap', that
# control respectively the type of marker used, their transparency and
# the colormap
plotting.scatter_matrix(data[['VIQ', 'MRI_Count', 'Height']],
                        c=(data['Gender'] == 'Female'), marker='o',
                        alpha=1, cmap='winter')

fig = plt.gcf()
fig.suptitle("blue: male, green: female", size=13)

plt.show()

# Linear models, multiple factors, and analysis of variance

## "formulas" to specify statistical models in Python

### A simple linear regression


In [ ]:
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Generate and show the data
x = np.linspace(-5, 5, 20)

# To get reproducible values, provide a seed value
np.random.seed(1)

y = -5 + 3*x + 4 * np.random.normal(size=x.shape)

# Plot the data
plt.figure(figsize=(5, 4))
plt.plot(x, y, 'o')

In [ ]:
#############################################################
# Multilinear regression model, calculating fit, P-values,
# confidence intervals etc.

# Convert the data into a Pandas DataFrame to use the formulas 
# framework in statsmodels
data = pandas.DataFrame({'x': x, 'y': y})

# Fit the model
model = ols("y ~ x", data).fit()

# Print the summary
print(model.summary())

In [ ]:
# Peform analysis of variance on fitted linear model
anova_results = anova_lm(model)

#print('\nANOVA results')
print(anova_results)

Statsmodels uses a statistical terminology: the `y` variable in
statsmodels is called 'endogenous' while the `x` variable is called
exogenous.  This is discussed in more detail in  
http://statsmodels.sourceforge.net/devel/endog_exog.html

To simplify, `y` (endogenous) is the value you are trying to predict,
while `x` (exogenous) represents the features you are using to make
the prediction.


Retrieve the estimated parameters from the model above. 


   **Hint**: use tab-completion to find the relevent attribute.


In [ ]:
# model.model.exog

## Categorical variables: comparing groups or multiple categories


In [ ]:
# remember, we modified "data" !
data = pandas.read_csv(csvfile, sep=';', na_values=".")

In [ ]:
model = ols("VIQ ~ Gender + 1", data)
modelfit = model.fit()
print(modelfit.summary())

In [ ]:
model.exog_names

**Forcing categorical**:

the 'Gender' is automatically detected as a
categorical variable, and thus each of its different values are
treated as different entities.

An integer column can be forced to be treated as categorical using:

    model = ols('VIQ ~ C(Gender)', data).fit()


**Intercept**:

We can remove the intercept using `- 1` in the formula,
or force the use of an intercept using `+ 1`.


By default, statsmodels treats a categorical variable with K possible
values as K-1 'dummy' boolean variables (the last level being
absorbed into the intercept term).  This is almost always a good
default choice - however, it is possible to specify different
encodings for categorical variables
(http://statsmodels.sourceforge.net/devel/contrasts.html).



**Link to t-tests between different FSIQ and PIQ**

In [ ]:
# To compare different types of IQ, we need to create a "long-form"
# table, listing IQs, where the type of IQ is indicated by a
# categorical variable::

data_fisq = pandas.DataFrame({'iq': data['FSIQ'], 'type': 'fsiq'})
data_piq = pandas.DataFrame({'iq': data['PIQ'], 'type': 'piq'})
data_long = pandas.concat((data_fisq, data_piq))

print(data_long)

#some_rows = (np.random.random_sample(data_long.shape[0]) < .15)
#print(data_long.loc[some_rows, 'type'])

In [ ]:
model = ols("iq ~ type", data_long).fit()
print(model.summary())

In [ ]:
# same values 
stats.ttest_ind(data['PIQ'], data['FSIQ']) 


 Going back to the brain size + IQ data, test if the VIQ of male and
 female are different after removing the effect of brain size, height
   and weight.
```

In [ ]:
data = pandas.read_csv(csvfile, sep=';', na_values=".")
data.columns

In [ ]:
modelfit = ols("VIQ ~ Weight + Height + MRI_Count + Gender", data).fit()
modelfit.summary()

In [ ]:
modelfit = ols("VIQ ~ Weight + Height + MRI_Count*Gender - 1", data).fit()
modelfit.summary()

In [ ]:
print(modelfit.f_test([1, -1, 0, 0, 0, 0]))
print(modelfit.f_test([[0, 0, 1, 0, 0, 0],[0, 0, 0, 1, 0, 0]]))  

In [ ]:
#modelfit.model.exog

# Robust statistics with statsmodel

In [ ]:
import statsmodels.api as sm
#data = sm.datasets.stackloss.load()
data = modelfit.model
huber_t = sm.RLM(data.endog, data.exog, M=sm.robust.norms.HuberT())
hub_results = huber_t.fit()
print(hub_results.params)
print(hub_results.bse)
print(hub_results.summary(yname='y', xname=data.exog_names))
#           xname=['var_{:d}'.format(i) for i in range(len(hub_results.params))]))

In [ ]:
nsample = 50
x1 = np.linspace(0, 20, nsample)
X = np.asarray([x1, (x1-5)**2]).T
X = sm.add_constant(X)
sig = 0.3   # smaller error variance makes OLS<->RLM contrast bigger
beta = [5, 0.5, -0.0]
y_true2 = np.dot(X, beta)
y2 = y_true2 + sig*1. * np.random.normal(size=nsample)
y2[[39,41,43,45,48]] -= 5   # add some outliers (10% of nsample

In [ ]:
modelfit = sm.OLS(y2, X).fit()
print(modelfit.params)
print(modelfit.bse)
# print(modelfit.predict() - y2)

In [ ]:
modelfit_rlm = sm.RLM(y2, X).fit()
print(modelfit_rlm.params)
print(modelfit_rlm.bse)

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
ax.plot(x1, y2, 'o',label="data")
ax.plot(x1, y_true2, 'b-', label="True")
predict_std, conf_lower, conf_upper = wls_prediction_std(modelfit)

ax.plot(x1, modelfit.fittedvalues, 'r-', label="OLS")
ax.plot(x1, conf_upper, 'r--')
ax.plot(x1, conf_lower, 'r--')
ax.plot(x1, modelfit_rlm.fittedvalues, 'g.-', label="RLM")
ax.legend(loc="best")

In [ ]:
#wls_prediction_std?

$\newcommand{L}[1]{\| #1 \|}\newcommand{VL}[1]{\L{ \vec{#1} }}\newcommand{R}[1]{\operatorname{Re}\,(#1)}\newcommand{I}[1]{\operatorname{Im}\, (#1)}$

# Correlated regressors

Load and configure libraries:

In [ ]:
from __future__ import print_function
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt
# Make numpy print 4 significant digits for prettiness
np.set_printoptions(precision=4, suppress=True)
# Seed random number generator
np.random.seed(42)

In [ ]:
%matplotlib inline

Imagine we have a TR (image) every 2 seconds, for 30 seconds. Here are
the times of the TR onsets, in seconds:

In [ ]:
times = np.arange(0, 30, 2)
times

Now we make a function returning an HRF shape for an input vector of
times:

In [ ]:
# Gamma distribution from scipy
from scipy.stats import gamma

In [ ]:
def spm_hrf(times):
    """ Return values for SPM-like HRF at given times """
    # Make output vector
    values = np.zeros(len(times))
    # Only evaluate gamma above 0 (undefined at <= 0)
    valid_times = times[times > 0]
    # Gamma pdf for the peak
    peak_values = gamma.pdf(valid_times, 6)
    # Gamma pdf for the undershoot
    undershoot_values = gamma.pdf(valid_times, 12)
    # Combine them, put back into values vector
    values[times > 0] = peak_values - 0.35 * undershoot_values
    # Scale area under curve to 1
    return values / np.sum(values)

Sample the HRF at the given times (to simulate an event starting at time
0), and at times - 2 (simulating an event starting at time 2):

In [ ]:
hrf1 = spm_hrf(times)
hrf2 = spm_hrf(times - 2) # An HRF with 2 seconds (one TR) delay
hrf1 = (hrf1 - hrf1.mean()) # Rescale and mean center
hrf2 = (hrf2 - hrf2.mean())
plt.plot(times, hrf1)
plt.plot(times, hrf2)

The Pearson correlation coefficient between the HRFs for the two events:

In [ ]:
np.corrcoef(hrf1, hrf2)

Make a signal that comes from the combination of the two HRFs:

In [ ]:
signal = hrf1 + hrf2
plt.plot(hrf1, label='hrf1')
plt.plot(hrf2, label='hrf2')
plt.plot(signal, label='signal (combined hrfs)')
plt.legend()

We’re going to make our simulated data from taking the signal (the two HRFs)
and adding some random noise:

In [ ]:
noise = np.random.normal(size=times.shape)
Y = signal + noise
plt.plot(times, signal)
plt.plot(times, Y, '+')

We are going to model this simulated signal in several different ways.  First,
we make a model that only has the first HRF as a regressor (plus a column of
ones to model the mean of the data):

In [ ]:
X_one = np.vstack((hrf1, np.ones_like(hrf1))).T
plt.imshow(X_one, interpolation='nearest', cmap='gray')
plt.title('Model with first HRF regressor only')

Next we make a model where we also include the second HRF as a regressor:

In [ ]:
X_both = np.vstack((hrf1, hrf2, np.ones_like(hrf1))).T
plt.imshow(X_both, interpolation='nearest', cmap='gray')
plt.title('Model with both HRF regressors')

Now we make a very large number of data vectors, each with the signal
(both HRFs) plus a different vector of noise.

In [ ]:
T = len(times)
iters = 10000
# Make 10000 Y vectors (new noise for each colum)
noise_vectors = np.random.normal(size=(T, iters))
# add signal to make data vectors
Ys = noise_vectors + signal[:, np.newaxis]
Ys.shape

We first fit the model with only the first HRF regressor to every (signal +
noise) sample vector.

In [ ]:
# Fit X_one to signals + noise
B_ones = npl.pinv(X_one).dot(Ys)

Next fit the model with both HRFs as regressors:

In [ ]:
# Fit X_both to signals + noise
B_boths = npl.pinv(X_both).dot(Ys)

Remember that the students-t statistic is:

$$
t = \frac{c^T \hat\beta}{\sqrt{\mathrm{var}(c^T \hat\beta)}}
$$

which works out to:

$$
t = \frac{c^T \hat\beta}{\sqrt{\hat{\sigma}^2 c^T (X^T X)^+ c}}
$$

where $\hat{\sigma}^2$ is our estimate of variance in the residuals, and
$(X^T X)^+$ is the [pseudo-inverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_pseudoinverse) of
$X^T X$.

That’s the theory. So, what is the distribution of the estimates we get for
the first beta, in the single-HRF model?

In [ ]:
plt.hist(B_ones[0], bins=50)
print(np.std(B_ones[0]))

The standard deviation of the estimates is what we observe. Does this
match what we would predict from the t-statistic formula above?

In [ ]:
C_one = np.array([1, 0])[:, None]  # column vector
sig_ones = np.sqrt(C_one.T.dot(npl.pinv(X_one.T.dot(X_one)).dot(C_one)))[0][0]
print(sig_ones)

Notice that the mean of the estimates, is somewhere above one, even
though we only added 1 times the first HRF as the signal:

In [ ]:
print(np.mean(B_ones[0]))
t_ones = B_ones/sig_ones
t_ones.mean()

This is because the single first regresssor has to fit *both* the first HRF in
the signal, and as much as possible of the second HRF in the signal, because
there is nothing else in the model to fit the second HRF shape.

What estimates do we get for the first regressor, when we have both regressors
in the model?

In [ ]:
plt.hist(B_boths[0], bins=50)
print(np.mean(B_boths[0]), np.std(B_boths[0]))

Two things have happened now we added the second (correlated) hrf2 regressor.
First, the mean of the parameter for the hrf1 regressor has dropped to 1,
because hrf1 is no longer having to model the signal from the second HRF.
Second, the variability of the estimate has increased.  This is what the
bottom half of the t-statistic predicts:

In [ ]:
C_both = np.array([1, 0, 0])[:, None]  # column vector
sig_both = np.sqrt(C_both.T.dot(npl.pinv(X_both.T.dot(X_both)).dot(C_both)))[0][0]

print(np.mean(B_boths[0]), np.std(B_boths[0]))

t_both_0 = B_boths[0]/sig_both
t_both_0.mean()

The estimate of the parameter for hrf2 has a mean of around 1, like the
parameter estimates for hrf1. This is what we expect because we have 1 x hrf1
and 1 x hrf2 in the signal. Not surprisingly, the hrf2 parameter estimate has
a similar variability to that for the hrf1 parameter estimate:

In [ ]:
plt.hist(B_boths[1], bins=50)
print(np.mean(B_boths[1]), np.std(B_boths[1]))

In [ ]:
C_both_1 = np.array([0, 1, 0])[:, None]  # column vector
np.sqrt(C_both_1.T.dot(npl.pinv(X_both.T.dot(X_both)).dot(C_both_1)))

The parameter estimates for hrf1 and hrf2 are anti-correlated:

In [ ]:
# Relationship of estimated parameter of hrf1 and hrf2
plt.plot(B_boths[0], B_boths[1], '.')
np.corrcoef(B_boths[0], B_boths[1])

## Orthogonalizing hrf2 with respect to hrf1

hrf2 is correlated with hrf1. That means that we can split up hrf2 into two
vectors, one being a multiple of hrf1, and the other being the remaining
unique contribution of hrf2. The sum of the two vectors is the original hrf2
regressor. Like this:

In [ ]:
# Regress hrf2 against hrf1 to get best fit of hrf2 using just hrf1
y = hrf2
X = hrf1[:, np.newaxis]  # hrf1 as column vector
B_hrf1_in_hrf2 = npl.pinv(X).dot(y)  # scalar multiple of hrf1 to best fit hrf2
hrf1_in_hrf2 = X.dot(B_hrf1_in_hrf2)  # portion of hrf2 that can be explained by hrf1
unique_hrf2 = hrf2 - hrf1_in_hrf2  # portion of hrf2 that cannot be explained by hrf1
plt.plot(times, hrf1, label='hrf1')
plt.plot(times, hrf2, label='hrf2')
plt.plot(times, hrf1_in_hrf2, label='hrf1 in hrf2')
plt.plot(times, unique_hrf2, label='hrf2 orth wrt hrf1')
plt.legend()
# hrf1 part of hrf2, plus unique part, equals original hrf2
np.allclose(hrf2, hrf1_in_hrf2 + unique_hrf2)

How much of the first regressor did we find in the second regressor?

In [ ]:
B_hrf1_in_hrf2

When we have the model with both hrf1 and hrf2, we are effectively multiplying
both parts of hrf2 by the same beta parameter, to fit the data. That is, we
are applying the same scaling to the part of hrf2 that is the same shape as
hrf1 and the part of hrf2 that cannot be formed from the hrf1 shape.

Now, what happens if we replace hrf2, by just the part of hrf2, that cannot be
explained by hrf1? Our second regressor is now hrf2 *orthogonalized with
respect to* hrf1:

In [ ]:
X_both_o = np.vstack((hrf1, unique_hrf2, np.ones_like(hrf1))).T
plt.imshow(X_both_o, interpolation='nearest', cmap='gray')

In [ ]:
plt.plot(times, X_both_o[:,0], times, X_both_o[:,1])

What will happen when we fit this model?

In [ ]:
B_boths_o = npl.pinv(X_both_o).dot(Ys)
# Distribution of parameter for hrf1 in orth model
plt.hist(B_boths_o[0], bins=50)
print(np.mean(B_boths_o[0]), np.std(B_boths_o[0]))

In [ ]:
# Predicted variance of hrf1 parameter is the same as for the
# model with hrf1 on its own
np.sqrt(C_both.T.dot(npl.pinv(X_both_o.T.dot(X_both_o)).dot(C_both)))

The parameter for the hrf1 regressor has now returned to the same value and
variance as it had when hrf1 was the only regressor in the model (apart from
the mean). For the orthogonalized model, we removed the part of hrf2 that
could be explained by hrf1. Now, the amount of hrf1, that we could find in
hrf2, has been added back to the parameter for hrf1, in order to make the
fitted $\hat{y}$ values the same as for the model with both HRFs.

In [ ]:
np.mean(B_boths[0, :]) + B_hrf1_in_hrf2

The hrf1 parameter in the orthogonalized model is the same as for the model
that only includes hrf1 - as if the orthogonalized hrf2 was not present. The
parameter for orthogonalized hrf2 is the same as the parameter for hrf2 in the
not-orthogonalized model. We still need the same amount of the *orthogonal
part* of the second regressor to explain the signal:

In [ ]:
# Example parameters from the single model
B_ones[:,:5]

In [ ]:
# Example parameters from the non-orth model
B_boths[:,:5]

In [ ]:
# Example parameters from the orth model
B_boths_o[:,:5]

In [ ]:
# The parameter for the hrf1 regressor in the orth model
# is the same as the parameter for the hrf1 regressor in the
# single regressor model
plt.plot(B_ones[0], B_boths_o[0], '.')
np.allclose(B_ones[0], B_boths_o[0])

In [ ]:
# The parameter for the orthogonalized hrf2 regressor is the same as the
# parameter for the non-orthogonalize hrf2 regressor in the
# non-orthogonalized model
plt.plot(B_boths[1], B_boths_o[1], '.')
np.allclose(B_boths[1], B_boths_o[1])

In [ ]:
# The parameter for the hrf1 regressor in the non-orth model
# is correlated with the parameter for the hrf1 regressor
# in the orth model.
plt.plot(B_boths[0], B_boths_o[0], '.')
np.corrcoef(B_boths[0], B_boths_o[0])

In [ ]:
# Relationship of estimated parameters for hrf1 and orthogonalized hrf2
# (they should be independent)
plt.plot(B_boths_o[0], B_boths_o[1], '+')
np.corrcoef(B_boths_o[0], B_boths_o[1])